# Visualizing differences - recreating fiji code
##Sep 2017

- retrieve info from original images + segmentations from kmeans + srm  and overlay
- save it to picture
- save animation

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import color, io, img_as_ubyte, external
from skimage.transform import resize
from skimage.filters import threshold_otsu, sobel
from skimage.morphology import closing, square, remove_small_objects

In [4]:
minSize = 50
#methList = ['kmeans','srm','pmrf']
path = "/Users/dani/Dropbox/prog/Apps_CIP/JSR_2017/daniTestMSM/examples/gambier_region/";
#path = "/Users/dani/Dropbox/prog/Apps_CIP/JSR_2017/daniTestMSM/examples/numbers/"

ic_orig = io.ImageCollection(path+"orig/*.tif")
ic_gt = io.ImageCollection(path+"groundtruth/*.tif")
ic_res_kmeans = io.ImageCollection(path+"orig/res/kmeans/tif/*.tif")
ic_res_srm = io.ImageCollection(path+"orig/res/srm/tif/*.tif")

ic_list = [ic_gt, ic_res_kmeans, ic_res_srm] 
#TODO: (1) what if gt not available?  (2) what if the processed slices are smaller than slices available

#define size of the output
im = ic_orig[0] 
h, w = im.shape
fh = h/400 #anim size
x=round(h/fh);
y=round(w/fh);

In [17]:
new_list = [ len(x) for x in ic_list]
minLen = min(new_list)
print(minLen)

3


In [6]:
#for each image, for each method, paint the edges accordingly
for n, curr_slice in enumerate(ic_orig[:minLen]):    
    im_orig = curr_slice 
    im_orig = img_as_ubyte(im_orig) 
    im_orig= resize(im_orig,(x,y),mode='reflect')
    im_orig = color.grey2rgb(im_orig, alpha=None)
    
    im = img_as_ubyte(ic_gt[n]) #change this 
    im = resize(im,(x,y),mode='reflect')
    thresh = threshold_otsu(im)  #needed due to interpolation crap
    im = closing(im > thresh, square(3))
    im = remove_small_objects(im, minSize) 
    edg = sobel(im)
    edg = edg >0
    im_orig[edg, 0] = 1
    #if darker borders
    im_orig[edg, 1] = 0
    im_orig[edg, 2] = 0    
    
    im = img_as_ubyte(ic_res_kmeans[n]) #change this 
    im = resize(im,(x,y),mode='reflect')
    thresh = threshold_otsu(im)  #needed due to interpolation crap
    im = closing(im > thresh, square(3))
    im = remove_small_objects(im, minSize) 
    edg = sobel(im)
    edg = edg >0
    im_orig[edg, 1] = 1
    #if darker borders
    im_orig[edg, 0] = 0
    im_orig[edg, 2] = 0    
    
    im = img_as_ubyte(ic_res_srm[n]) #change this 
    im = resize(im,(x,y),mode='reflect')
    thresh = threshold_otsu(im)
    im = closing(im > thresh, square(3))
    im = remove_small_objects(im, minSize) 
    edg = sobel(im)
    edg = edg >0
    im_orig[edg, 2] = 1
    #if darker borders
    im_orig[edg, 0] = 0
    im_orig[edg, 1] = 0    
    
    #saving slices
    path = ic_list[1].files[n] #any index larger than 0 carries segmentation
    pathPieces=path.split('kmeans')
    pathPieces2=pathPieces[-1].split('/tif/')
    filename=pathPieces[0]+'vis/tif/'+pathPieces2[-1]
    #print(filename)
    #io.imsave(filename, im_orig) #There's problem with the format being save, file seems to be corrupted
    im_orig = img_as_ubyte(im_orig)
    external.tifffile.imsave(filename, im_orig,photometric='rgb')

/Users/dani/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/dani/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/dani/anaconda/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [ ]:
plt.imshow(im_orig)

In [ ]:
#gif animation
import os
os.system('imagemagick-converting-command filein fileout')